In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-8uz4e1_v
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-8uz4e1_v
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0d2ab99cccbbc682722e708515fe9c4cfc50185a
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4716 sha256=9418a56cbb707684c4eef2d81481e71c3366275f077bbbd092e918f3b23116f4
  Stored in directory: /tmp/pip-ephem-wheel-cache-ptur0szn/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin


created output directory at /content/src
Out bin /content/result.out


In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [3]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git


  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-4q2xrd_a
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-4q2xrd_a
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0d2ab99cccbbc682722e708515fe9c4cfc50185a
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4716 sha256=d392aa0948dc9dfbe5aa8d02f3f3ad48bfe0144c7c1184691152d7f2c458494f
  Stored in directory: /tmp/pip-ephem-wheel-cache-1_vvq383/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [4]:
%load_ext nvcc_plugin


created output directory at /content/src
Out bin /content/result.out


In [36]:
%%cu

#include<iostream>
#include<vector>
#define size 1000000
__global__ void VectorAdd(int* a,int *b,int *c,int N)
{
    int t_x= (blockIdx.x*blockDim.x)+ threadIdx.x;
    if(t_x<N) c[t_x]=a[t_x]+b[t_x];
}
void reference(std::vector<int>& a, std::vector<int>& b,std::vector<int>& c) {
           cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
cudaEventRecord(start);
    for (int i = 0; i <size; i++)
     c[i]= a[i] + b[i];
            cudaEventRecord(stop);
         cudaEventSynchronize(stop);
    float elapsed_time_ms;
    cudaEventElapsedTime(&elapsed_time_ms, start, stop);
           std::cout<<"CPU EXECUTION:"<<elapsed_time_ms<<"\n";
}
int main()
{
     int N=size;
     size_t bytes=sizeof(int)*N;

    std::vector<int> a;
    a.reserve(N);
    std::vector<int> b;
    b.reserve(N);
    std::vector<int> c;
    c.reserve(N);


    for(int i=0;i<N;i++)
    {
        a.push_back(rand()%100);
        b.push_back(rand()%100);
    }

    int *d_a,*d_b,*d_c;

    cudaMalloc(&d_a,bytes);
    cudaMalloc(&d_b,bytes);
    cudaMalloc(&d_c,bytes);

    cudaMemcpy(d_a,a.data(),bytes,cudaMemcpyHostToDevice);
    cudaMemcpy(d_b,b.data(),bytes,cudaMemcpyHostToDevice);

    int blocksize = 1<<10;
    int gridsize = (N+blocksize-1)/blocksize;
       cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
cudaEventRecord(start);
    VectorAdd<<<gridsize,blocksize>>>(d_a,d_b,d_c,N);
 cudaEventRecord(stop);
         cudaEventSynchronize(stop);
    float elapsed_time_ms;
    cudaEventElapsedTime(&elapsed_time_ms, start, stop);

    cudaMemcpy(c.data(),d_c,bytes,cudaMemcpyDeviceToHost);
      for(int i=0;i<50;i++)
     {
         std::cout<<c[i]<<"\t";
     }
     std::cout<<"\n";
     std::cout<<"GPU_EXECUTION:"<<elapsed_time_ms<<"\n";
     std::cout<<"\n";
     reference(a,b,c);
    std::cout<<"COMPLETED";
    return 0;


}

169	92	128	178	70	89	149	89	66	108	79	96	112	85	102	31	80	136	149	53	102	40	121	122	85	39	171	129	132	177	30	111	41	75	70	119	127	81	98	93	95	154	172	54	153	92	144	51	112	133	
GPU_EXECUTION:0.203808

CPU EXECUTION:7.39229
COMPLETED


In [29]:
!nvidia-smi


Sun Jan 14 17:19:13 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--